In [1]:
from stylometry import *
import pymongo
import time
import warnings
warnings.filterwarnings('ignore')

import spacy
import en_core_web_lg
nlp = en_core_web_lg.load()

In [2]:
## connect the database to my virtual machine
client = pymongo.MongoClient('mongodb://localhost/')
db = client['gutenberg_db']
collection = db['gutenberg_collection']

In [3]:
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'gutenberg_db'), 'gutenberg_collection')

In [4]:
text = collection.find({'file': 'project_gutenberg/51185.txt'}, {'text': 1})[0]['text']

In [5]:
s = time.time()
doc = nlp(text)
tokens = [token.orth_.lower() for token in doc if not token.is_punct and token if not token.is_stop]
sentences = [sent.string.strip() for sent in doc.sents]
print(f'Took {time.time() - s}s to process')

Took 1.4822821617126465s to process


In [6]:
sttr(tokens)

0.627

In [7]:
hapax_legomenon(tokens)

0.3534591194968553

In [8]:
yules_k(tokens)

1703.7019456860444

In [9]:
function_words(doc)

0.32435685271551074

In [10]:
avg_sentence_length_word(sentences)

8.441368078175897

In [11]:
avg_sentence_length_chars(sentences)

48.504885993485345

In [12]:
avg_syllables_per_word(tokens)

1.7941299790356393

In [13]:
punctuation_sentence(sentences)

2.6351791530944624

In [14]:
shannon_entropy(tokens)

9.51165465635602

In [15]:
simpsons_d(tokens)

0.9968982595359701

In [16]:
average_nps(doc)

220.33333333333334

In [17]:
noun_to_verb(doc)

0.5780346820809249

In [18]:
noun_to_adj(doc)

0.8141112618724559

In [19]:
verb_to_adv(doc)

0.6985645933014354

In [20]:
avg_dependency_distance(doc)

2.0813625343073197

In [21]:
get_style_metrics('project_gutenberg/51185.txt')

NameError: name 'time' is not defined